In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import os, sys # For filepaths
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import label_binarize
import pickle
import sklearn.preprocessing as pre

In [ ]:
''' Add the datasets and libraries to the system path '''

# Find the path to our implementations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
home_directory = os.path.dirname(parent_directory)
libraries_path = os.path.join(home_directory, 'Libraries')

# Find the path to the datasets
datasets_path = os.path.join(home_directory, 'Datasets')

# Add them both to the system path
sys.path.append(datasets_path)
sys.path.append(libraries_path)

In [ ]:
''' Load the dataset '''

dataset = np.load(os.path.join(datasets_path, 'fashion_train.npy'))
print("Dataset shape: ", dataset.shape)

In [ ]:
test_dataset = np.load(os.path.join(datasets_path, 'fashion_test.npy'))

In [ ]:
X_train = dataset[:, :-1]  # Define the images as all columns except the last one
y_train = dataset[:, -1] # Define the labels as the last column

In [ ]:
X_test = test_dataset[:, :-1]  # Define the images as all columns except the last one
y_test = test_dataset[:, -1] # Define the labels as the last column

In [ ]:
# scaling the images
scaler = pre.StandardScaler()
X = scaler.fit_transform(X_train)